In [ ]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests from optkeras.optkeras import OptKeras
import optkeras
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop

import optuna
from zipfile import ZipFile

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *

#Downloading dataset
zipurl = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip'    
r = requests.get(zipurl, stream = True) 
  
with open("dataset.zip","wb") as pdf: 
    for chunk in r.iter_content(chunk_size=1024): 
         if chunk: 
            pdf.write(chunk)
            
# Extract all the contents of zip file in current directory            
with ZipFile('dataset.zip', 'r') as zipObj:
    zipObj.extractall()

In [1]:
from optkeras.optkeras import OptKeras
import optkeras
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop

import optuna

ModuleNotFoundError: No module named 'optkeras'

In [2]:
!sudo pip3 install --upgrade pip

Requirement already up-to-date: pip in /usr/local/lib/python3.6/site-packages (20.2.2)


In [3]:
!sudo pip3 install  keras==2.1.5 tensorflow numpy pandas pillow opencv-python sklearn 

     |████████████████████████████████| 334 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 320.4 MB 34 kB/s s eta 0:00:01     |█████████████████████████████▍  | 294.4 MB 34.3 MB/s eta 0:00:01     |███████████████████████████████▊| 317.2 MB 22.9 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 37.1 MB/s eta 0:00:01    |███████                         | 3.2 MB 37.1 MB/s eta 0:00:01
     |████████████████████████████████| 10.5 MB 33.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 40.8 MB/s eta 0:00:01
     |████████████████████████████████| 49.4 MB 31.4 MB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 38.4 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 8.4 MB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 9.1 MB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 30.3 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.6 MB/s  eta 0:00:01
     |██████████████████████████

In [4]:
!sudo pip3 install keras==2.1.5 tensorflow==1.13.1 numpy pandas pillow sklearn optuna scikit-image ray ray[tune] 

     |████████████████████████████████| 92.5 MB 18.7 MB/s eta 0:00:01
     |████████████████████████████████| 226 kB 46.1 MB/s eta 0:00:01
     |████████████████████████████████| 12.4 MB 27.4 MB/s eta 0:00:01
     |████████████████████████████████| 22.0 MB 44.3 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 367 kB 23.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 21.8 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 76 kB 8.6 MB/s  eta 0:00:01
     |████████████████████████████████| 1.3 MB 25.5 MB/s eta 0:00:01
     |████████████████████████████████| 68 kB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 11.6 MB 31.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB

     |████████████████████████████████| 110 kB 30.2 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 33.3 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.0 MB/s  eta 0:00:01
     |████████████████████████████████| 88 kB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 8.5 MB/s  eta 0:00:01
     |████████████████████████████████| 460 kB 23.8 MB/s eta 0:00:01
     |████████████████████████████████| 148 kB 46.6 MB/s eta 0:00:01
     |████████████████████████████████| 257 kB 25.1 MB/s eta 0:00:01
     |████████████████████████████████| 100 kB 13.8 MB/s ta 0:00:01
     |████████████████████████████████| 98 kB 11.9 MB/s eta 0:00:01
  Created wheel for optuna: filename=optuna-2.0.0-py3-none-any.whl size=312964 sha256=0287975d0cde5d65fe16ec05f0e042bc070518599eba35832e96fb53e21cf854
  Stored in directory: /root/.cache/pip/wheels/6a/1f/c0/e68cc4a2c4088c8498110b5af910a0fe2679c758

  Created wheel for contextvars: filename=contextvars-2.4-py3-none-any.whl size=7665 sha256=a976d3b961ddeeb86c7d7b1a9b0b61c19811f46351b13237238a63c8690151a9
  Stored in directory: /root/.cache/pip/wheels/41/11/53/911724983aa48deb94792432e14e518447212dd6c5477d49d3
Successfully built optuna alembic gpustat PrettyTable nvidia-ml-py3 psutil idna-ssl pyperclip contextvars
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [5]:
!sudo pip3 install optkeras

In [6]:
!sudo pip3 install pandas

In [7]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from glob import glob
from zipfile import ZipFile

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *


#Replica Catalog
rc = ReplicaCatalog()
input_files = glob('*.jpg')
input_files.sort()
in_files=[]

for file in input_files:
    in_files.append(File(file))
    rc.add_replica("local", File(file), str(Path(".").resolve() / file))
    
rc.write()


#Transformation
pre_process_resize = Transformation( "preprocess1.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/preprocess1.py",
        is_stageable=True)

pre_process_augment = Transformation( "Augmentation.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Augmentation.py",
        is_stageable=True)

data_split  = Transformation( "Data_Split.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Data_Split.py",
        is_stageable=True)

vgg_model  = Transformation( "VGG_model.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/VGG_model.py",
        is_stageable=True)

hpo =  Transformation( "Optuna.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Optuna.py",
        is_stageable=True)
                    
tc = TransformationCatalog()\
    .add_transformations(pre_process_resize,pre_process_augment,data_split,vgg_model,hpo)\
    .write()

#Workflow
wf = Workflow("Cats_and_Dogs", infer_dependencies=True)


resized_images = File('resized_images.txt')
all_files = [File("resized_{}".format(f.lfn)) for f in in_files]
labels = File('labels.txt')

job_preprocess1 = Job(pre_process_resize)\
                    .add_inputs(*in_files)\
                    .add_outputs(*all_files,resized_images,labels) 

aug_images_txt = File('augmentation.txt')
aug_labels_txt = File('aug_labels.txt')
augmented_files = []
for f in all_files:
    augmented_files.extend([File(str(f).replace("{}".format(os.path.splitext(str(f))[0]), "Aug_{}_{}".format(os.path.splitext(str(f))[0],i))) for i in range(3)])

    
job_preprocess2 = Job(pre_process_augment)\
                    .add_inputs(*all_files,labels)\
                    .add_outputs(aug_images_txt,aug_labels_txt,*augmented_files)

training_data = File('training.pkl')
testing_data = File('testing.pkl')
val_data = File('validation.pkl')

job_data_split = Job(data_split)\
                    .add_inputs(*augmented_files,labels)\
                    .add_outputs(training_data,testing_data,val_data)

model = File('model.h5')

job_vgg_model = Job(vgg_model)\
                    .add_inputs(*augmented_files,training_data,testing_data,val_data)\
                    .add_outputs(model)
model1 = File('model_hpo.h5')
optuna_csv = File("Optuna_log_file.csv")

job_hpo = Job(hpo)\
                    .add_args(optuna_csv)\
                    .add_inputs(model,*augmented_files,training_data,testing_data,val_data)\
                    .add_outputs(model1,optuna_csv)


wf.add_jobs(job_preprocess1,job_preprocess2,job_data_split,job_vgg_model,job_hpo)                                    

In [8]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.08.27 03:17:21.054 UTC:
2020.08.27 03:17:21.060 UTC:   -----------------------------------------------------------------------
2020.08.27 03:17:21.065 UTC:   File for submitting this DAG to HTCondor           : Cats_and_Dogs-0.dag.condor.sub
2020.08.27 03:17:21.071 UTC:   Log of DAGMan debugging messages                 : Cats_and_Dogs-0.dag.dagman.out
2020.08.27 03:17:21.076 UTC:   Log of 

[##################################################] 100.0% ..Success (Completed: 29, Queued: 0, Running: 0, Failed: 0)



####################
# pegasus-analyzer #
####################
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

Submit Directory   : /home/scitech/shared-data/CatsAndDogs/scitech/pegasus/Cats_and_Dogs/run0021
Total jobs         :     29 (100.00%)
# jobs succeeded   :     29 (100.00%)
# jobs failed      :      0 (0.00%)
# jobs held        :      0 (0.00%)
# jobs unsubmitted :      0 (0.00%)



######################
# pegasus-statistics #
######################
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workf

In [ ]:

         


aug_images_txt = File('augmentation.txt')
aug_labels_txt = File('aug_labels.txt')

job_preprocess2 = Job(pre_process_augment)\
                    .add_inputs(labels,resized_images)\
                    .add_outputs(aug_images_txt,aug_labels_txt)


In [ ]:
augmented_files = []
for f in all_files:
    augmented_files.extend([File(str(f).replace("{}".format(os.path.splitext(str(f))[0]), "Aug_{}_{}".format(os.path.splitext(str(f))[0],i))) for i in range(3)])

In [ ]:
len(augmented_files)

In [ ]:
len(all_files)

In [ ]:

pre_process_augment = Transformation( "Augmentation.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Augmentation.py",
        is_stageable=True)   


aug_images_txt = File('augmentation.txt')
aug_labels_txt = File('aug_labels.txt')
augmented_files = []
for f in all_files:
    augmented_files.extend([File(str(f).replace("{}".format(os.path.splitext(str(f))[0]), "Aug_{}_{}".format(os.path.splitext(str(f))[0],i))) for i in range(3)])

job_preprocess2 = Job(pre_process_augment)\
                    .add_inputs(*all_files,labels,resized_images)\
                    .add_outputs(aug_images_txt,aug_labels_txt,*augmented_files)
